# Integration Tests Demo

This notebook demonstrates key integration tests for the full analysis pipeline.

## Test Categories

1. **Main Analysis Integration**: Full workflow from data loading to HDF5 saving
2. **Phase Analysis Integration**: Signal stacking, STFT ridge, CWT phase reconstruction
3. **Full Pipeline**: End-to-end analysis with spectrum, density, and plotting

## Usage

Run each cell sequentially to explore the integration test functionality.


In [ ]:
# Setup
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from unittest.mock import Mock, patch

# Add project root to path
project_root = Path().resolve().parent.parent.parent
sys.path.insert(0, str(project_root))

from ifi.analysis.main_analysis import run_analysis
from ifi.analysis.phase_analysis import SignalStacker, STFTRidgeAnalyzer
from ifi.db_controller.nas_db import NAS_DB
from ifi.db_controller.vest_db import VEST_DB

print("Setup complete!")


## 1. Main Analysis Integration Test

### Test: Full Workflow (Mock Data)


In [ ]:
# Create synthetic test data
fs = 50e6
duration = 0.01
t = np.arange(0, duration, 1/fs)
n_samples = len(t)

# Create DataFrame with multiple channels
test_data = pd.DataFrame({
    'TIME': t,
    'CH0': np.sin(2 * np.pi * 8e6 * t),
    'CH1': np.sin(2 * np.pi * 8e6 * t + 0.1),
    'CH2': np.sin(2 * np.pi * 8e6 * t + 0.2),
})
test_data.set_index('TIME', inplace=True)

print(f"Test data shape: {test_data.shape}")
print(f"Time range: {t[0]:.6f} to {t[-1]:.6f} seconds")
print(f"Sampling frequency: {fs/1e6:.1f} MHz")


## 2. Phase Analysis Integration Test

### Test: Signal Stacker


In [ ]:
# Test SignalStacker
fs = 50e6
f0 = 8e6
duration = 0.01
t = np.arange(0, duration, 1/fs)

# Create reference and probe signals
ref_signal = np.sin(2 * np.pi * f0 * t)
probe_signal = np.sin(2 * np.pi * f0 * t + 0.1)

# Test SignalStacker
stacker = SignalStacker(fs)
f0_detected = stacker.find_fundamental_frequency(ref_signal)

print(f"Expected frequency: {f0/1e6:.2f} MHz")
print(f"Detected frequency: {f0_detected/1e6:.2f} MHz")
print(f"Error: {abs(f0_detected - f0)/f0 * 100:.2f}%")
print(f"Test passed: {abs(f0_detected - f0) < f0 * 0.1}")
